In [1]:
!nvidia-smi

Sat Oct  2 15:50:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!unzip /content/drive/MyDrive/RealTime_Face_Detector/augmentedDataset.zip

Archive:  /content/drive/MyDrive/RealTime_Face_Detector/augmentedDataset.zip
   creating: /content/content/dataset/
   creating: /content/content/dataset/trainImages/
   creating: /content/content/dataset/trainImages/not_me/
  inflating: /content/content/dataset/trainImages/not_me/Abdullah_Gul_0006.jpg  
  inflating: /content/content/dataset/trainImages/not_me/Aaron_Sorkin_0001.jpg  
  inflating: /content/content/dataset/trainImages/not_me/Abdullah_Gul_0012.jpg  
  inflating: /content/content/dataset/trainImages/not_me/Aaron_Peirsol_0001.jpg  
  inflating: /content/content/dataset/trainImages/not_me/Abdoulaye_Wade_0002.jpg  
  inflating: /content/content/dataset/trainImages/not_me/Abdullah_Gul_0019.jpg  
  inflating: /content/content/dataset/trainImages/not_me/Abdullah_0003.jpg  
  inflating: /content/content/dataset/trainImages/not_me/Aaron_Guiel_0001.jpg  
  inflating: /content/content/dataset/trainImages/not_me/Aaron_Patterson_0001.jpg  
  inflating: /content/content/dataset/trainIm

In [3]:
# Common Imports
import os 
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

%matplotlib inline
sns.set()

In [4]:
# Tensorflow imports
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
# Dataset Information
train_aug_image_folder = os.path.join('dataset','face_dataset_train_aug_image')
train_image_folder = os.path.join('dataset','trainImages')
test_image_folder = os.path.join('dataset','testImages')
img_height, img_width = 250,250
num_classes = 2

validation_ratio=0.2
batch_size=20
AUTOTUNE=tf.data.AUTOTUNE

Creation of Dataset

In [23]:
# Train and validation sets of initial dataset
train_ds = keras.preprocessing.image_dataset_from_directory(
    train_image_folder,
    validation_split=validation_ratio,
    subset="training",
    seed=42,
    image_size=(img_height,img_width),
    label_mode='categorical',
    batch_size=batch_size,
    shuffle=True    
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    train_image_folder,
    validation_split=validation_ratio,
    subset="validation",
    seed=42,
    image_size=(img_height,img_width),
    label_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

Found 122 files belonging to 2 classes.
Using 98 files for training.
Found 122 files belonging to 2 classes.
Using 24 files for validation.


In [42]:
# Train and validation sets of augmented dataset
train_aug_ds = keras.preprocessing.image_dataset_from_directory(
    train_aug_image_folder,
    seed=42,
    image_size=(img_height,img_width),
    label_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

# val_aug_ds = keras.preprocessing.image_dataset_from_directory(
#     train_aug_image_folder,
#     validation_split=validation_ratio,
#     subset="validation",
#     seed=42,
#     image_size=(img_height, img_width),
#     batch_size=batch_size,
#     label_mode='categorical',
#     shuffle=True)

Found 700 files belonging to 1 classes.


In [25]:
test_ds = keras.preprocessing.image_dataset_from_directory(
    test_image_folder,
    image_size=(img_height,img_width),
    label_mode='categorical',
    shuffle=True
)

Found 18 files belonging to 2 classes.


In [26]:
class_names = test_ds.class_names
class_names

['me', 'not_me']

Building Model with Transfer Learning

VGG16

In [27]:
base_model = keras.applications.vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(img_height,img_width,3))
# Set layers to non-trainable
for layer in base_model.layers:
  layer.trainable=False

# Add custom layers on top of the convolutional layers of VGG16
flatten = keras.layers.Flatten()(base_model.output)
dense_4096_1 = keras.layers.Dense(4096,activation='relu')(flatten)
dense_4096_2 = keras.layers.Dense(4096,activation='relu')(dense_4096_1)
output = keras.layers.Dense(num_classes,activation='sigmoid')(dense_4096_2)

VGG16 = keras.models.Model(inputs=base_model.input, outputs=output,name='VGG16')
VGG16.summary()

Model: "VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0     

ResNet50

In [28]:
base_model = keras.applications.ResNet50(weights='imagenet',include_top=False,input_shape=(img_height,img_width,3))

# Set layer to non-trainable
for layer in base_model.layers:
  layer.trainable=False

# Add custom layers on top of ResNet50
global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(num_classes,activation='sigmoid')(global_avg_pooling)

ResNet50 = keras.models.Model(inputs=base_model.input,outputs=output,name='ResNet50')
ResNet50.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 256, 256, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 125, 125, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 125, 125, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

ResNet152

In [29]:
base_model=keras.applications.ResNet152(weights='imagenet',include_top=False,input_shape=(img_height,img_width,3))

# Set layers to non-trainable
for layer in base_model.layers:
  layer.trainable=False

# Add custom layers on top of ResNet
global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(num_classes,activation='sigmoid')(global_avg_pooling)

ResNet152 = keras.models.Model(inputs=base_model.input,outputs=output,name='ResNet152')
ResNet152.summary()

Model: "ResNet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 256, 256, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 125, 125, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 125, 125, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

Xception

In [30]:
base_model = keras.applications.Xception(weights='imagenet',include_top=False,input_shape=(img_height,img_width,3))

# Set layers to non-trainable
for layer in base_model.layers:
  layer.trainable=False

# Add custom layers on top of Xception
global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(num_classes,activation='sigmoid')(global_avg_pooling)

Xception = keras.models.Model(inputs=base_model.input,outputs=output,name='Xception')
Xception.summary()

Model: "Xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 124, 124, 32) 864         input_9[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 124, 124, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 124, 124, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

MobileNet

In [31]:
base_model = keras.applications.MobileNet(weights='imagenet',include_top=False,input_shape=(img_height,img_width,3))

# set layers to non-trainable
for layer in base_model.layers:
  layer.trainable=False

# Adding custom layer on top of MobileNet
global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(num_classes,activation='sigmoid')(global_avg_pooling)

MobileNet = keras.models.Model(inputs=base_model.input,outputs=output,name='MobileNet')
MobileNet.summary()

Model: "MobileNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 125, 125, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 125, 125, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 125, 125, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 125, 125, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 125, 125, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 125, 125, 32)      0 

Training

1. VGG16
2. ResNet50
3. ResNet152
4. Xception
5. MobileNet



In [32]:
face_classifier = MobileNet
face_classifier.summary()

name_to_save = f"models/face_classifier_{face_classifier.name}_aug.h5"

Model: "MobileNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 125, 125, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 125, 125, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 125, 125, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 125, 125, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 125, 125, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 125, 125, 32)      0 

In [64]:
# ModelCheckpoint to save model in case of interrupting the learning process
checkpoint = ModelCheckpoint(name_to_save,
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

# EarlyStopping to find best model with a large number of epochs
earlystop = EarlyStopping(monitor='val_loss',
                          restore_best_weights=True,
                          patience=100,
                          verbose=1)
callbacks=[earlystop,checkpoint]

In [49]:
face_classifier.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.1),metrics=['accuracy'])

In [50]:
epochs=500

In [52]:
history=face_classifier.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds)
face_classifier.save(name_to_save)

Epoch 1/500
5/5 [==============================] - 2s 153ms/step - loss: 4.4477 - accuracy: 0.7653 - val_loss: 0.5719 - val_accuracy: 0.9583

Epoch 00001: val_loss improved from inf to 0.57191, saving model to models/face_classifier_MobileNet_aug.h5
Epoch 2/500
5/5 [==============================] - 1s 53ms/step - loss: 1.8255 - accuracy: 0.9184 - val_loss: 5.9524 - val_accuracy: 0.7917

Epoch 00002: val_loss did not improve from 0.57191
Epoch 3/500
5/5 [==============================] - 1s 53ms/step - loss: 1.4205 - accuracy: 0.9184 - val_loss: 1.6079 - val_accuracy: 0.9167

Epoch 00003: val_loss did not improve from 0.57191
Epoch 4/500
5/5 [==============================] - 1s 54ms/step - loss: 0.4589 - accuracy: 0.9286 - val_loss: 0.0298 - val_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.57191 to 0.02976, saving model to models/face_classifier_MobileNet_aug.h5
Epoch 5/500
5/5 [==============================] - 1s 55ms/step - loss: 0.0377 - accuracy: 0.9898 - val_loss: 1.9

In [53]:
model_name = 'face_classifier_MobileNet_aug.h5'
face_classifier = keras.models.load_model(f'models/{model_name}')

In [54]:
def test_image_classifier_with_folder(model,path,y_true,img_height=250,img_width=250,class_names=['me','not_me']):
  num_classes=len(class_names)
  total=0
  correct=0

  for filename in os.listdir(path):
    test_path = os.path.join(path,filename)
    test_image = keras.preprocessing.image.load_img(test_path,target_size=(img_height,img_width,3))
    test_image = keras.preprocessing.image.img_to_array(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    result = model.predict(test_image)

    y_pred = class_names[np.array(result[0]).argmax(axis=0)]
    iscorrect = 'correct' if y_pred == y_true else 'incorrect'
    print('{}-{}'.format(iscorrect,filename))
    for index in range(num_classes):
      print("\t{:6} with probability of {:.2f}%".format(class_names[index],result[0][index]*100))
    total+=1
    if y_pred == y_true:
      correct+=1
  print("\nTotal accuracy is {:.2f}%={}/{} samples classified correctly".format(correct/total*100,correct,total))

In [55]:
test_image_classifier_with_folder(face_classifier,'dataset/testImages/me',y_true='me')

correct-IMG_0004.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004113291.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-18f3ca8e-f3d6-4020-812b-1800b5b67c15~2.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
incorrect-FB_IMG_1543004107327.jpg
	me     with probability of 0.71%
	not_me with probability of 99.70%
correct-a8e5e4f3-4b4b-4d22-ade3-1acd6c74bc10.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004173806.jpg
	me     with probability of 63.94%
	not_me with probability of 20.53%
correct-FB_IMG_1543004123855.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-IMG_0077.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
incorrect-IMG_0007.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%

Total accuracy is 77.78%=7/9 samples classified correctly


In [58]:
test_image_classifier_with_folder(face_classifier,'dataset/testImages/not_me',y_true='not_me')

correct-Aicha_El_Ouafi_0002.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-Alan_Ball_0001.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-Akbar_Hashemi_Rafsanjani_0001.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-Aicha_El_Ouafi_0001.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-Akbar_Hashemi_Rafsanjani_0002.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-Aitor_Gonzalez_0002.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-Akbar_Hashemi_Rafsanjani_0003.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
incorrect-Aitor_Gonzalez_0001.jpg
	me     with probability of 75.57%
	not_me with probability of 11.88%
incorrect-Aicha_El_Ouafi_0003.jpg
	me     with probability of 78.73%
	not_me with probability of 49.89%

Total accuracy is 77.78%=7/9 samples classified corr

In [60]:
face_classifier = Xception
face_classifier.summary()

name_to_save = f"models/face_classifier_{face_classifier.name}_aug.h5"

face_classifier.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.1),metrics=['accuracy'])
epochs=500
history=face_classifier.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds)
face_classifier.save(name_to_save)

Model: "Xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 124, 124, 32) 864         input_9[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 124, 124, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 124, 124, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [61]:
model_name = 'face_classifier_Xception_aug.h5'
face_classifier = keras.models.load_model(f'models/{model_name}')
test_image_classifier_with_folder(face_classifier,'dataset/testImages/me',y_true='me')
test_image_classifier_with_folder(face_classifier,'dataset/testImages/not_me',y_true='not_me')

correct-IMG_0004.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
incorrect-FB_IMG_1543004113291.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
incorrect-18f3ca8e-f3d6-4020-812b-1800b5b67c15~2.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-FB_IMG_1543004107327.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-a8e5e4f3-4b4b-4d22-ade3-1acd6c74bc10.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
incorrect-FB_IMG_1543004173806.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-FB_IMG_1543004123855.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
incorrect-IMG_0077.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-IMG_0007.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%

Total accuracy is 55.56%=5/9 samples classified correctly
correct-

In [62]:
face_classifier = ResNet152
face_classifier.summary()

name_to_save = f"models/face_classifier_{face_classifier.name}_aug.h5"

face_classifier.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.1),metrics=['accuracy'])
epochs=500
history=face_classifier.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds)
face_classifier.save(name_to_save)

Model: "ResNet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 256, 256, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 125, 125, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 125, 125, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 6/500
5/5 [==============================] - 1s 168ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00006: val_loss improved from 0.00000 to 0.00000, saving model to models/face_classifier_MobileNet_aug.h5
Epoch 7/500
5/5 [==============================] - 1s 171ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00007: val_loss did not improve from 0.00000
Epoch 8/500
5/5 [==============================] - 1s 170ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00008: val_loss did not improve from 0.00000
Epoch 9/500
5/5 [==============================] - 1s 169ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00009: val_loss did not improve from 0.00000
Epoch 10/500
5/5 [==============================] - 1s 169ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_ac

In [63]:
model_name = 'face_classifier_ResNet152_aug.h5'
face_classifier = keras.models.load_model(f'models/{model_name}')
test_image_classifier_with_folder(face_classifier,'dataset/testImages/me',y_true='me')
test_image_classifier_with_folder(face_classifier,'dataset/testImages/not_me',y_true='not_me')

correct-IMG_0004.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004113291.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-18f3ca8e-f3d6-4020-812b-1800b5b67c15~2.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004107327.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-a8e5e4f3-4b4b-4d22-ade3-1acd6c74bc10.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004173806.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004123855.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-IMG_0077.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-IMG_0007.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%

Total accuracy is 100.00%=9/9 samples classified correctly
correct-Aicha_E

In [65]:
face_classifier = ResNet50
face_classifier.summary()

name_to_save = f"models/face_classifier_{face_classifier.name}_aug.h5"

face_classifier.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.1),metrics=['accuracy'])
epochs=500
history=face_classifier.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds)
face_classifier.save(name_to_save)

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 256, 256, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 125, 125, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 125, 125, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/500
5/5 [==============================] - 1s 89ms/step - loss: 1.7989 - accuracy: 0.9388 - val_loss: 4.9221e-06 - val_accuracy: 1.0000

Epoch 00002: val_loss improved from 1.83708 to 0.00000, saving model to models/face_classifier_ResNet152_aug.h5
Epoch 3/500
5/5 [==============================] - 1s 92ms/step - loss: 5.9622e-04 - accuracy: 1.0000 - val_loss: 0.3374 - val_accuracy: 0.9583

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/500
5/5 [==============================] - 1s 88ms/step - loss: 0.7403 - accuracy: 0.9796 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.00000 to 0.00000, saving model to models/face_classifier_ResNet152_aug.h5
Epoch 5/500
5/5 [==============================] - 1s 91ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00005: val_loss did not improve from 0.00000
Epoch 6/500
5/5 [==============================] - 1s 91ms/step - loss: 0.0000e+00 - accur

In [66]:
model_name = 'face_classifier_ResNet50_aug.h5'
face_classifier = keras.models.load_model(f'models/{model_name}')
test_image_classifier_with_folder(face_classifier,'dataset/testImages/me',y_true='me')
test_image_classifier_with_folder(face_classifier,'dataset/testImages/not_me',y_true='not_me')

correct-IMG_0004.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004113291.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-18f3ca8e-f3d6-4020-812b-1800b5b67c15~2.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
incorrect-FB_IMG_1543004107327.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%
correct-a8e5e4f3-4b4b-4d22-ade3-1acd6c74bc10.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004173806.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004123855.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-IMG_0077.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
incorrect-IMG_0007.jpg
	me     with probability of 0.00%
	not_me with probability of 100.00%

Total accuracy is 77.78%=7/9 samples classified correctly
correct-Aich

In [67]:
face_classifier = VGG16
face_classifier.summary()

name_to_save = f"models/face_classifier_{face_classifier.name}_aug.h5"

face_classifier.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.1),metrics=['accuracy'])
epochs=500
history=face_classifier.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds)
face_classifier.save(name_to_save)

Model: "VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0     

In [68]:
model_name = 'face_classifier_VGG16_aug.h5'
face_classifier = keras.models.load_model(f'models/{model_name}')
test_image_classifier_with_folder(face_classifier,'dataset/testImages/me',y_true='me')
test_image_classifier_with_folder(face_classifier,'dataset/testImages/not_me',y_true='not_me')

correct-IMG_0004.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004113291.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-18f3ca8e-f3d6-4020-812b-1800b5b67c15~2.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004107327.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-a8e5e4f3-4b4b-4d22-ade3-1acd6c74bc10.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004173806.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-FB_IMG_1543004123855.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-IMG_0077.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%
correct-IMG_0007.jpg
	me     with probability of 100.00%
	not_me with probability of 0.00%

Total accuracy is 100.00%=9/9 samples classified correctly
correct-Aicha_E

In [69]:
!zip -r /content/models.zip /content/models

  adding: content/models/ (stored 0%)
  adding: content/models/face_classifier_ResNet50_aug.h5 (deflated 8%)
  adding: content/models/face_classifier_MobileNet_aug.h5 (deflated 8%)
  adding: content/models/face_classifier_ResNet152_aug.h5 (deflated 8%)
  adding: content/models/face_classifier_Xception_aug.h5 (deflated 8%)
  adding: content/models/face_classifier_VGG16_aug.h5 (deflated 21%)
